# Inference from any pose

With this notebook you can run the inference of ContourCraft using an arbitrary (for example, non-SMPL(-X)) mesh sequence.

To run it, you will need a mesh sequence as a `.pkl` file containing a dictionary of the following items (see `DEFAULTS.data_root/examples/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, V, 3]
* `faces`: np.array [F, 3]

You will also need a garment mesh (as `.obj` file) which is aligned with the first frame of your body sequence.

In this notebook we first show how to convert a garment mesh in `.obj` format into a garment dict used in ContourCraft. 

Then, we show how to use the configuration file `aux/from_any_pose.yaml` to run inference over an arbitrary mesh sequence.

## Create template file from the .obj file:

Use `utils.mesh_creation::obj2template()` function to convert an `.obj` file into a template dictionary and then save it with `pickle_dump`

`DEFAULTS.data_root/examples/fromanypose/skirt.obj` is provided as an example

In [1]:
from utils.mesh_creation import obj2template
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path

from utils.show import write_video 
from aitviewer.headless import HeadlessRenderer

/var/folders/sh/1lh6p1hx46n_8jppf9dnjmth0000gn/T/tmp6crc22i0


In [2]:
obj_path = Path(DEFAULTS.data_root) / 'examples' / 'fromanypose' / 'skirt.obj'
out_template_path = Path(DEFAULTS.data_root) / 'examples' / 'fromanypose' / 'skirt.pkl'

template_dict = obj2template(obj_path, verbose=True)
pickle_dump(template_dict, out_template_path)

Adding coarse edges... (may take a while)
Done.


In [3]:
from utils.mesh_creation import add_pinned_verts


pinned_indices = \
[2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215,
 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235,
 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2327, 2328, 2351, 2492, 2497, 2669, 2670, 2671, 2674, 2837, 3139,
 3200, 3204, 3359, 3362, 3495, 3512, 3634, 3638, 3805, 3965, 3967, 4133, 4137, 4140, 4335, 4340, 4506, 4509, 4669, 4674,
 4749, 4812, 4849, 4853, 5138, 5309, 5342, 5469, 5474, 5503, 5646, 5650, 5654, 5855, 5857, 6028, 6091, 6204, 6209, 6280,
 6374, 6377, 6378, 6473, 6649, 6654, 6814, 6817, 6986, 6989, 6990, 6992, 7172, 7178, 7336, 7500, 7505, 7661, 7665, 7666]

add_pinned_verts(out_template_path, pinned_indices)

## Inference with a mesh sequence

Here we use an sequence of arbitrary meshes.  The garment template needs to be aligned with the first frame of the sequence.

The mesh sequence has to be a `.pkl` file containing a dictionary of the following items (see `DEFAULTS.data_root/examples/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, V, 3]
* `faces`: np.array [F, 3]

### Load a runner and a dataloader from `from_any_pose` config

In [4]:
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch
from utils.arguments import create_runner

In [5]:
# use HOOD
# modules, config = load_params('aux/from_any_pose_hood')
# checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'hood_final.pth'

# use ContourCraft
modules, config = load_params('aux/from_any_pose')
checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'contourcraft.pth'


runner_module, runner, aux_modules = create_runner(modules, config)

# NOCUDA fix
# state_dict =  torch.load(checkpoint_path)
state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))

runner.load_state_dict(state_dict['training_module'])

Warp 1.10.0 initialized:
   CUDA not enabled in this build
   Devices:
     "cpu"      : "arm"
   Kernel cache:
     /Users/jan.rojc/Library/Caches/warp/1.10.0


<All keys matched successfully>

### Create a dataloader

Same as in the previous part, set several variables to specify which garment and which pose sequence you want to use:


- `pose_sequence_path`: path to the smpl pose sequence relative to `DEFAULTS.data_root`. For this example we use `examples/fromanypose/mesh_sequence.pkl`
- `garment_template_path`: path to the .pkl file with the garment template. For this example we use `examples/fromanypose/skirt.pkl`

In [6]:
from utils.datasets import make_fromanypose_dataloader

pose_sequence_path = Path('examples') / 'fromanypose' / 'mesh_sequence.pkl'
# garment_template_path = Path('examples') / 'fromanypose' / 'skirt.pkl'
# pose_sequence_path = Path('examples') / 'fromanypose' / 'mesh_sequence.pkl'
garment_template_path = Path('aux_data') / 'garment_dicts' / 'smpl' / 'aaron_009::bottom.pkl'


dataloader = make_fromanypose_dataloader(pose_sequence_type='mesh', 
                       pose_sequence_path=pose_sequence_path, 
                       garment_template_path=garment_template_path)

### load sample, infer and save trajectories

To visualise the saved trajectories, see `Inference.ipynb::write_video`

In [ ]:
sample = next(iter(dataloader))

# NOCUDA fix
# sample = move2device(sample, 'cuda:0')
# runner.to('cuda:0')
sample = move2device(sample, 'cpu')
runner.to('cpu')

trajectories_dict = runner.valid_rollout(sample, n_steps=100)

# Save the sequence to disk
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_m.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

In [ ]:
# Careful!: creating more that one renderer in a single session causes an error
renderer = HeadlessRenderer()

In [ ]:
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_m.pkl'
out_video = Path(DEFAULTS.data_root) / 'temp' / 'output.mp4'
write_video(out_path, out_video, renderer)

Rendering frames: 100%|██████████| 204/204 [00:05<00:00, 36.97it/s]


Video saved to /Users/jan.rojc/Documents/MagCode/Data/ccraft_data/temp/output.mp4


# Testing

In [ ]:
next(iter(dataloader))